# How to execute large jobs?: Using Batch Job

Most of the simple, basic openEO usage examples show synchronous downloading of results: you submit a process graph with a (HTTP POST) request and receive the result as direct response of that same request. This only works properly if the processing doesn’t take too long (order of seconds, or a couple of minutes at most).

For the heavier work (larger regions of interest, larger time series, more intensive processing, …) you have to use batch jobs.

This notebook shows how to programmatically create and interact with batch job using the openEO Python client library.

In [2]:
import openeo

# connect to the backend and authenticate
connection = openeo.connect(url = "openeo-staging.creo.vito.be")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo-staging.creo.vito.be/openeo/1.1/' with OidcBearerAuth>

In [3]:
# load your data collection
cube = connection.load_collection(
                            "SENTINEL2_L2A",
                            bands = ["B04", "B03", "B02"],
                            temporal_extent = ("2022-05-01", "2022-05-30"),
                            spatial_extent = {'west': 3.202609,'south': 51.189474,'east': 3.254708,'north': 51.204641,'crs': 'EPSG:4326'},
                            max_cloud_cover=80

)
cube.max_time()

In [4]:
# Store raster data as GeoTIFF files
cube = cube.save_result(format="GTiff")

While not necessary, it is also recommended to give your batch job a descriptive title so it’s easier to identify in your job listing.

In [5]:
job = cube.create_job(title = "Testing batch job in openeo")

The job object returned by create_job() is a BatchJob object. It is basically a client-side reference to a batch job that exists on the back-end and allows to interact with that batch job

Starting a batch job is pretty straightforward with the `start_job()` or if you want to monitor it you can also use `start_and_wait()` and itprint some progress messages on the way for you.

In [6]:
result = job.start_and_wait()

0:00:00 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': send 'start'
0:00:11 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': created (progress N/A)
0:00:16 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': created (progress N/A)
0:00:23 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': created (progress N/A)
0:00:31 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': created (progress N/A)
0:00:41 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': created (progress N/A)
0:00:53 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': running (progress N/A)
0:01:08 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': running (progress N/A)
0:01:27 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': running (progress N/A)
0:01:51 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': running (progress N/A)
0:02:21 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': running (progress N/A)
0:02:59 Job 'j-aae1404e0e27402f9ffe6a3a1bc28c77': finished (progress N/A)


A batch job on a back-end is fully identified by its job_id.

In [7]:
job.job_id

'j-aae1404e0e27402f9ffe6a3a1bc28c77'

Depending on your situation or use case: make sure to properly take note of the batch job id. It allows you to “reconnect” to your job on the back-end, even if it was created at another time, by another script/notebook or even with another openEO client. Then you can later use use `Connection.job("your job id")` to create a BatchJob object for an existing batch job.

A batch job typically takes some time to finish, and you can check its status with the `status()` method.

In [8]:
job.status()

'finished'

Batch job logs can be fetched with `job.logs()`

In [9]:
job.logs()

[{'id': '[1683633680243, 4169147]',
  'time': '2023-05-09T12:01:20.243Z',
  'level': 'info',
  'message': "Starting job 'j-aae1404e0e27402f9ffe6a3a1bc28c77' from user User('3e24e251-2e9a-438f-90a9-d4500e576574', {'oidc_userinfo': {'sub': '3e24e251-2e9a-438f-90a9-d4500e576574', 'email_verified': True, 'name': 'Pratichhya Sharma', 'preferred_username': 'pratichhya.sharma@vito.be', 'given_name': 'Pratichhya', 'family_name': 'Sharma', 'email': 'pratichhya.sharma@vito.be'}}) (proxy user None)"},
 {'id': '[1683633680309, 4170651]',
  'time': '2023-05-09T12:01:20.309Z',
  'level': 'debug',
  'message': 'job_options: {}'},
 {'id': '[1683633681157, 4180813]',
  'time': '2023-05-09T12:01:21.157Z',
  'level': 'info',
  'message': "Dry run extracted these source constraints: [(('load_collection', ('SENTINEL2_L2A', (('eo:cloud_cover', (('lte', 80),)), ('productType', (('eq', 'L2A'),))))), {'temporal_extent': ('2022-05-01', '2022-05-30'), 'spatial_extent': {'west': 3.202609, 'south': 51.189474, 'east': 3.254708, 'north': 51.204641, 'crs': 'EPSG:4326'}, 'bands': ['B04', 'B03', 'B02'], 'properties': {'productType': {'process_graph': {'asc': {'process_id': 'eq', 'arguments': {'x': {'from_parameter': 'value'}, 'y': 'L2A'}, 'result': True}}}, 'eo:cloud_cover': {'process_graph': {'lte1': {'process_id': 'lte', 'arguments': {'x': {'from_parameter': 'value'}, 'y': 80}, 'result': True}}}}})]"},
 {'id': '[1683633681738, 4182145]',
  'time': '2023-05-09T12:01:21.738Z',
  'level': 'info',
  'message': 'mapped job_id j-aae1404e0e27402f9ffe6a3a1bc28c77 to application ID job-aae1404e0e-3e24e251-2e9a-438f-9'},
 {'id': '[1683633691564, 4345]',
  'time': '2023-05-09T12:01:31.564Z',
  'level': 'info',
  'message': 'batch_job.py main: start 2023-05-09 12:01:31.564262'},
 {'id': '[1683633691564, 4653]',
  'time': '2023-05-09T12:01:31.564Z',
  'level': 'info',
  'message': "batch_job.py argv: ['/opt/openeo/lib64/python3.8/site-packages/openeogeotrellis/deploy/batch_job.py', '/batch_jobs/j-aae1404e0e27402f9ffe6a3a1bc28c77/job_specification.json', '/batch_jobs/j-aae1404e0e27402f9ffe6a3a1bc28c77', 'out', 'stdout', 'job_metadata.json', '1.1.0', '[]', '3e24e251-2e9a-438f-90a9-d4500e576574', '0.0']"},
 {'id': '[1683633691564, 5241]',
  'time': '2023-05-09T12:01:31.564Z',
  'level': 'info',
  'message': 'batch_job.py pid 69; ppid 15; cwd /opt/spark/work-dir'},
 {'id': '[1683633691775, 5558]',
  'time': '2023-05-09T12:01:31.775Z',
  'level': 'warning',
  'message': "batch_job.py version info {'date': '2023-05-09T12:01:31Z', 'versions': {'openeo': '0.16.0', 'openeo_driver': '0.47.0a1.dev20230504+500', 'openeo-geopyspark': '0.11.0a1.dev20230506+1253'}}"},
 {'id': '[1683633691776, 6012]',
  'time': '2023-05-09T12:01:31.776Z',
  'level': 'info',
  'message': "creating job dir PosixPath('/batch_jobs/j-aae1404e0e27402f9ffe6a3a1bc28c77') (parent dir: {'path': '/batch_jobs', 'mode': 'drwxr-xr-x', 'uid': 18585, 'user': 'spark', 'gid': 18585, 'group': 'spark', 'size': 6}))"},
 {'id': '[1683633691776, 6536]',
  'time': '2023-05-09T12:01:31.776Z',
  'level': 'info',
  'message': 'Using temp dir /opt/spark/work-dir/tmp'},
 {'id': '[1683633691877, 6889]',
  'time': '2023-05-09T12:01:31.877Z',
  'level': 'info',
  'message': "Trying to load 'custom_processes' with PYTHONPATH ['/opt/venv/lib/python3.8/site-packages/openeogeotrellis/deploy', '/usr/local/spark/python/lib/pyspark.zip', '/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip', '/usr/local/spark-3.4.0/jars/spark-core_2.12-3.4.0.jar', '/opt/spark/work-dir/$PYTHONPATH', '/opt/tensorflow/python38/2.3.0', '/opt/openeo/lib/python3.8/site-packages', '/usr/lib64/python38.zip', '/usr/lib64/python3.8', '/usr/lib64/python3.8/lib-dynload', '/usr/local/lib64/python3.8/site-packages', '/usr/local/lib/python3.8/site-packages', '/usr/lib64/python3.8/site-packages', '/usr/lib/python3.8/site-packages']"},
 {'id': '[1683633691877, 7818]',
  'time': '2023-05-09T12:01:31.877Z',
  'level': 'info',
  'message': '\'custom_processes\' not loaded: Module

Once a batch job is finished you can get a handle to the results (which can be a single file or multiple files) and metadata with `get_results()`.
The result metadata describes the spatio-temporal properties of the result and is in fact a valid STAC item.

In [10]:
results = result.get_results()
results.get_metadata()

{'assets': {'openEO_2022-05-03Z.tif': {'eo:bands': [{'center_wavelength': 0.6645,
     'name': 'B04'},
    {'center_wavelength': 0.56, 'name': 'B03'},
    {'center_wavelength': 0.4966, 'name': 'B02'}],
   'file:nodata': [0],
   'href': 'https://openeo-staging.creo.vito.be/openeo/1.1/jobs/j-aae1404e0e27402f9ffe6a3a1bc28c77/results/assets/M2UyNGUyNTEtMmU5YS00MzhmLTkwYTktZDQ1MDBlNTc2NTc0/aff72f67c43208d79b576d8451f6bde7/openEO_2022-05-03Z.tif?expires=1684238747',
   'roles': ['data'],
   'title': 'openEO_2022-05-03Z.tif',
   'type': 'image/tiff; application=geotiff'},
  'openEO_2022-05-08Z.tif': {'eo:bands': [{'center_wavelength': 0.6645,
     'name': 'B04'},
    {'center_wavelength': 0.56, 'name': 'B03'},
    {'center_wavelength': 0.4966, 'name': 'B02'}],
   'file:nodata': [0],
   'href': 'https://openeo-staging.creo.vito.be/openeo/1.1/jobs/j-aae1404e0e27402f9ffe6a3a1bc28c77/results/assets/M2UyNGUyNTEtMmU5YS00MzhmLTkwYTktZDQ1MDBlNTc2NTc0/4c78b026d08dce60b943479e75e0e055/openEO_2022-05-08

In the general case, when you have one or more result files (also called “assets”), the easiest option to download them is using `download_files()` (plural) where you just specify a download folder (otherwise the current working directory will be used by default).

In [15]:
results.download_files("data/out")